### Process dataset

In [1]:
%load_ext jupyter_black

In [2]:
from transformers import DebertaV2Tokenizer, AutoTokenizer, DebertaV2ForMaskedLM
from transformers import PreTrainedTokenizerFast
from datasets import load_dataset, load_from_disk
from tqdm.auto import tqdm

c:\Users\Matheus\Documents\Git\Longformer-PT\venv_lg_former\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Load dataset

In [13]:
dataset_carolina = load_from_disk("../data/corpus-carolina")

In [14]:
dataset_carolina

DatasetDict({
    corpus: Dataset({
        features: ['meta', 'text'],
        num_rows: 2107045
    })
})

In [4]:
dataset_brwac = load_dataset("brwac", data_dir="../data/brwac")

Dataset brwac downloaded and prepared to C:/Users/Matheus/.cache/huggingface/datasets/brwac/default-data_dir=..%2Fdata%2Fbrwac/1.0.0/cd622d293b6b90355cb6cb412bd9673902d6b9ce11d05b096a509f3aa40a0317. Subsequent calls will reuse this data.


100%|██████████| 1/1 [00:06<00:00,  6.36s/it]


In [5]:
dataset_brwac.save_to_disk("../data/brwac_dataset")

In [22]:
dataset_brwac["train"][0]["text"]

{'paragraphs': [['Conteúdo recente'],
  ['ESPUMA MARROM CHAMADA "NINGUÉM MERECE"'],
  ['31 de Agosto de 2015, 7:07 , por paulo soavinski - | No one following this article yet.'],
  ['Visualizado 202 vezes'],
  ['JORNAL ELETRÔNICO DA ILHA DO MEL'],
  ['Uma espuma marrom escuro tem aparecido com frequência na Praia de Fora.',
   'Na faixa de areia ela aparece disseminada e não chama muito a atenção.',
   'No Buraco do Aipo, com muitas pedras, ela aparece concentrada.',
   'É fácil saber que esta espuma estranha está lá, quando venta.',
   'Pequenos algodões de espuma começam a flutuar no espaço, pertinho da Praia do Saquinho.',
   'Quem pode ajudar na coleta deste material, envio a laboratório renomado e pagamento de análises, favor entrar em contato com o site.']]}

In [39]:
ds_train = dataset_brwac.split("train[:1%]")

AttributeError: 'DatasetDict' object has no attribute 'split'

In [33]:
dataset_brwac["train"].map(
    lambda example: {"text": ["\n".join(sum(example["text"]["paragraphs"], []))]},
    num_proc=4,
    batched=True,
)

TypeError: list indices must be integers or slices, not str

In [12]:
"\n".join(sum(dataset_brwac["train"][0]["text"]["paragraphs"], []))

'Conteúdo recente\nESPUMA MARROM CHAMADA "NINGUÉM MERECE"\n31 de Agosto de 2015, 7:07 , por paulo soavinski - | No one following this article yet.\nVisualizado 202 vezes\nJORNAL ELETRÔNICO DA ILHA DO MEL\nUma espuma marrom escuro tem aparecido com frequência na Praia de Fora.\nNa faixa de areia ela aparece disseminada e não chama muito a atenção.\nNo Buraco do Aipo, com muitas pedras, ela aparece concentrada.\nÉ fácil saber que esta espuma estranha está lá, quando venta.\nPequenos algodões de espuma começam a flutuar no espaço, pertinho da Praia do Saquinho.\nQuem pode ajudar na coleta deste material, envio a laboratório renomado e pagamento de análises, favor entrar em contato com o site.'

### Train Tokenizer

In [40]:
import ftfy

In [41]:
def batch_iterator(dataset, batch_size=10_000):
    for i in tqdm(range(0, len(dataset), batch_size)):
        text = dataset[i : i + batch_size]["text"]
        text = [ftfy.fix_text(t) for t in text]
        yield text

In [42]:
tokenizer = AutoTokenizer.from_pretrained("microsoft/deberta-base")

In [44]:
new_tokenizer = tokenizer.train_new_from_iterator(
    batch_iterator(dataset_carolina['corpus']), vocab_size=tokenizer.vocab_size
)

  3%|▎         | 6/211 [01:56<1:01:22, 17.96s/it]

In [ ]:
new_tokenizer.save_pretrained("../models/deberta-ptbr-tokenizer-carolina")

### Load model and tokenizer

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("../models/deberta-ptbr-tokenizer")
model = DebertaV2ForMaskedLM.from_pretrained("microsoft/deberta-v3-base", vocab_size=tokenizer.vocab_size, ignore_mismatched_sizes=True)

### Splitting data by max_lenght

In [ ]:
MAX_POS_EMBD = model.config.max_position_embeddings
WINDOW = 200

In [ ]:
from functools import partial

def _tokenizer(sample, tokenizer, window, max_lenght):
    tkn_text = tokenizer(sample['text'], add_special_tokens=False)

    dev_train_dataset = []

    for sample in tkn_text['input_ids']:
        for i in range(0, len(sample), window):
            dev_train_dataset.append(tokenizer.decode(sample[i:max_lenght+i]).strip())
    return {'text': dev_train_dataset}

def _tokenizer_aux(sample, tokenizer):
    tkn_text = tokenizer(sample['text'], return_special_tokens_mask=True)
    return tkn_text

partial_tokenizer = partial(_tokenizer, tokenizer=tokenizer, window=WINDOW, max_lenght=MAX_POS_EMBD-2)
partial_tokenizer_aux = partial(_tokenizer_aux, tokenizer=tokenizer)

### Preprocess dataset

In [ ]:
dataset = dataset.map(partial_tokenizer, batched=True, num_proc=4)

In [ ]:
dataset.save_to_disk("../data/corpus-carolina-tokenized")

### Generate id's

In [ ]:
dataset = dataset.map(partial_tokenizer_aux, batched=True, num_proc=4)

In [ ]:
dataset.save_to_disk("../data/corpus-carolina-aux")